In [ ]:
from datasets import load_dataset

dataset = load_dataset("din0s/asqa")

sample = dataset['dev'][0]
q = sample['ambiguous_question']

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/t5-large-ssm-nq")

model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-large-ssm-nq")

In [ ]:
input_ids = tokenizer.encode(q, return_tensors="pt")
gen_output = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)

print(q)
tokenizer.decode(gen_output[0], skip_special_tokens=True)